# Analysis on Reddit posts for customer problems

## Importing libraries

In [6]:
import praw
from dotenv import load_dotenv
import os
import time

In [25]:
load_dotenv()

reddit_client_id = os.getenv("REDDIT_CLIENT_ID")
reddit_client_secret = os.getenv("REDDIT_CLIENT_SECRET")
user_agent = os.getenv("USER_AGENT")
gemini_api_key = os.getenv("GEMINI_API_KEY")

## Reddit scrapper

In [8]:
# inititlize reddit instance

reddit = praw.Reddit(
    client_id=reddit_client_id,
    client_secret=reddit_client_secret,
    user_agent=user_agent
)

In [10]:
#test reddit

for submission in reddit.subreddit("all").search("Meesho scam", limit=5):
    print(submission.title)

Meesho scam!! Stay safe y’all😭😭
Got scammed for the first time (meesho)
Evey 2nd Delivery boy in Delhi NCR are scammer Now Meesho Delivery Boy scammed me
Meesho paylater scam(sharing my experience)
How are they giving 8gb varient at just 15,499, Is meesho a Scam ?


In [18]:
query_list = ["Meesho scam", "Never order from Meesho", "Meesho problem", "Meesho fraud"]
posts = []
for query in query_list:
    for submission in reddit.subreddit("all").search(query, limit=40, sort="new"):
        posts.append({
            "title": submission.title,
            "text": submission.selftext,
            "url": submission.url,
            "score": submission.score
        })

print(f"Collected {len(posts)} posts")

Collected 158 posts


In [19]:
texts = [p["title"] + " " + p["text"] for p in posts if p["text"]]

## Analysis of post using gemini 1.5 flash

In [23]:
from openai import OpenAI

#Initialize openai client
gemini_client = OpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
    )

In [ ]:
batch_size = 10
all_summaries = []

for i in range(0, len(texts), batch_size):
    chunk = texts[i:i+batch_size]
    chunk_prompt = f"""
    You are analyzing user complaints about Meesho from Reddit.
    Extract the problems customers face and summarize them into categories such as:
    - Sellers: onboarding, catalog management, payouts, visibility or growth.
    - Customers: discovery, trust, checkout, delivery, post-purchase support.
    - Delivery partners: address accuracy, efficiency, payments last-mile reliability.
    - Scalability, trust or ecosystem-wide issue.

    List problems under each category. Here are the complaints:
    {chunk}
    """

    response = gemini_client.chat.completions.create(
        model="gemini-1.5-flash",
        messages=[{"role": "user", "content": chunk_prompt}]
    )
    summary = response.choices[0].message.content
    all_summaries.append(summary)
    time.sleep(1)

#aggregate results and create markdown summary
final_prompt = f"""
Combine these summaries of Meesho complaints and organize the problems under each specified category.
Format your response in markdown, using headers for each category and grouping problems under appropriate lists.

Here are the batch summaries:
{all_summaries}
"""

final_response = gemini_client.chat.completions.create(
    model="gemini-1.5-flash",
    messages=[{"role": "user", "content": final_prompt}]
)

markdown_summary = final_response.choices[0].message.content
with open("meesho_complaint_summary.md", "w") as f:
    f.write(markdown_summary)
